# 신용평가(credit scoring) 프로젝트

## 프로젝트 목표
- **신용등급 예측 모델**을 구축해 대출 심사 의사결정을 지원한다.
- 금융권 실무에서 자주 사용하는 **이진 분류** 문제를 다룬다.

## 사용 알고리즘
- Logistic Regression
- Random Forest
- XGBoost

## 금융권 업무 관점
- 신용평가는 **대출 승인/한도/금리 결정**의 핵심 근거
- 모델은 **연체 가능성(불량 위험)**을 사전에 추정해 리스크를 관리
- 해석 가능한 모델(Logistic)은 **규제 대응**에 유리, 트리/앙상블은 **성능 개선**에 유리


In [ ]:
# =============================================================================
# 1. 라이브러리 임포트
# =============================================================================
# 학습 목표: 전처리/모델/평가에 필요한 기본 라이브러리를 준비한다
# Java 비유: Maven/Gradle로 의존성을 추가하고 import 하는 단계와 유사

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    roc_auc_score,
)
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

# XGBoost는 설치되지 않았을 수 있으므로 예외 처리
try:
    import xgboost as xgb
except ModuleNotFoundError:
    xgb = None
    print("⚠️ xgboost가 설치되어 있지 않습니다. (conda install -c conda-forge xgboost)")


In [ ]:
# =============================================================================
# 2. 샘플 데이터 로드 또는 생성
# =============================================================================
# 학습 목표: 실제 데이터가 없을 때 대체 데이터를 생성해 흐름을 검증한다
# Java 비유: 개발 단계에서 Mock 데이터를 만들어 기능을 검증하는 것과 유사

from pathlib import Path

DATA_DIR = Path.cwd() / "data"
DATA_PATH = DATA_DIR / "cs-training.csv"  # Kaggle 'Give Me Some Credit' 원본 파일명 예시

if DATA_PATH.exists():
    try:
        # Kaggle 데이터 로드 (예: cs-training.csv)
        raw_df = pd.read_csv(DATA_PATH)
        print("✅ Kaggle 데이터 로드 완료")
    except (FileNotFoundError, pd.errors.EmptyDataError) as e:
        print(f"❌ 데이터 로드 오류: {e}")
        raw_df = None
else:
    raw_df = None
    print("⚠️ Kaggle 데이터가 없어 샘플 데이터를 생성합니다.")

# Kaggle 데이터가 없으면 make_classification으로 샘플 생성
if raw_df is None:
    # 특징: 나이, 소득, 부채비율, 연체 횟수 등을 가정
    X, y = make_classification(
        n_samples=5000,
        n_features=6,
        n_informative=4,
        n_redundant=1,
        weights=[0.85, 0.15],
        random_state=42,
    )

    sample_df = pd.DataFrame(
        X,
        columns=[
            "age",
            "monthly_income",
            "debt_ratio",
            "late_30_59",
            "late_60_89",
            "late_90_plus",
        ],
    )

    # 타겟: Good(0)/Bad(1)로 가정
    sample_df["target"] = y
    raw_df = sample_df

# 데이터프레임 통일: target 컬럼명 정리
if "SeriousDlqin2yrs" in raw_df.columns:
    # Kaggle 데이터의 타겟 컬럼명 정리
    raw_df = raw_df.rename(columns={"SeriousDlqin2yrs": "target"})

# 타겟을 Good/Bad로 표현하기 위한 매핑
raw_df["target_label"] = raw_df["target"].map({0: "Good", 1: "Bad"})

raw_df.head()


In [ ]:
# =============================================================================
# 3. 데이터 구조 확인
# =============================================================================
# 학습 목표: 데이터 형태/결측치/불균형 여부를 파악한다
# Java 비유: List<Map>의 size, key 타입, null 비율을 점검하는 단계와 유사

print("✅ 데이터 크기 (행, 열):", raw_df.shape)
print("\n✅ 데이터 타입:")
print(raw_df.dtypes)

print("\n✅ 결측치 현황:")
print(raw_df.isnull().sum())

print("\n✅ 타겟 분포 (Good/Bad):")
print(raw_df["target_label"].value_counts())

print("\n✅ 타겟 비율:")
print(raw_df["target_label"].value_counts(normalize=True).round(4))


## 탐색적 데이터 분석(EDA)

### 실무 관점 요약
- 금융권 신용평가에서 **소득, 부채비율, 연체 횟수**는 핵심 위험 지표로 활용된다.
- 이상치는 **부정확한 입력/이상 거래**를 의미할 수 있어 반드시 점검한다.
- 클래스 불균형(불량 비율이 낮음)은 **모델 편향**의 주요 원인이므로 확인이 필수다.


In [ ]:
# =============================================================================
# 4. 수치형 변수 분석 (통계 요약 / 분포 / 이상치)
# =============================================================================
# 학습 목표: 수치형 변수의 분포와 이상치를 파악한다
# Java 비유: 숫자형 필드의 기본 통계와 이상값 검증 로직과 유사

numeric_cols = raw_df.select_dtypes(include=["int64", "float64"]).columns.tolist()
if "target" in numeric_cols:
    numeric_cols.remove("target")

print("✅ 수치형 변수 목록:", numeric_cols)

# describe() 통계 요약
print("\n✅ 수치형 변수 통계 요약")
print(raw_df[numeric_cols].describe())

# 히스토그램 (분포 확인)
raw_df[numeric_cols].hist(bins=30, figsize=(12, 8))
plt.suptitle("수치형 변수 분포 (히스토그램)")
plt.tight_layout()
plt.show()

# 박스플롯 (이상치 탐지)
plt.figure(figsize=(12, 6))
sns.boxplot(data=raw_df[numeric_cols])
plt.title("수치형 변수 이상치 분포 (박스플롯)")
plt.xticks(rotation=45)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()


In [ ]:
# =============================================================================
# 5. 범주형 변수 분석
# =============================================================================
# 학습 목표: 범주형 변수의 분포를 파악한다
# Java 비유: enum/문자열 값의 빈도를 집계하는 단계와 유사

categorical_cols = raw_df.select_dtypes(include=["object", "category"]).columns.tolist()

if not categorical_cols:
    print("⚠️ 범주형 변수가 없습니다. (샘플 데이터 기준)")
else:
    print("✅ 범주형 변수 목록:", categorical_cols)
    for col in categorical_cols:
        print(f"\n✅ {col} 값 분포")
        print(raw_df[col].value_counts())

        plt.figure(figsize=(8, 4))
        sns.countplot(data=raw_df, x=col)
        plt.title(f"{col} 분포")
        plt.xticks(rotation=45)
        plt.grid(alpha=0.3)
        plt.tight_layout()
        plt.show()


In [ ]:
# =============================================================================
# 6. 타겟 변수 분석 (Good/Bad 비율)
# =============================================================================
# 학습 목표: 클래스 불균형 여부를 확인한다
# Java 비유: 정상/이상 케이스의 비율을 점검하는 단계와 유사

# Good/Bad 비율
target_counts = raw_df["target_label"].value_counts()
print("✅ 타겟 분포:")
print(target_counts)

plt.figure(figsize=(5, 4))
sns.countplot(data=raw_df, x="target_label")
plt.title("타겟 분포 (Good/Bad)")
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

imbalance_ratio = (target_counts.min() / target_counts.max()) if target_counts.max() > 0 else 0
print(f"✅ 클래스 불균형 비율(소수/다수): {imbalance_ratio:.2f}")


In [ ]:
# =============================================================================
# 7. 변수 간 관계 분석
# =============================================================================
# 학습 목표: 상관관계와 타겟별 분포 차이를 확인한다
# Java 비유: 여러 필드 간 상관/의존성을 확인하는 통계 분석 단계

# 수치형 변수 상관관계 히트맵
plt.figure(figsize=(8, 6))
correlation = raw_df[numeric_cols + ["target"]].corr()
sns.heatmap(correlation, annot=True, cmap="coolwarm", vmin=-1, vmax=1)
plt.title("수치형 변수 상관관계 히트맵")
plt.tight_layout()
plt.show()

# 타겟별 변수 분포 비교 (violinplot, boxplot)
if numeric_cols:
    selected_cols = numeric_cols[:3]  # 주요 변수 3개만 선택
    for col in selected_cols:
        plt.figure(figsize=(6, 4))
        sns.violinplot(data=raw_df, x="target_label", y=col)
        plt.title(f"타겟별 {col} 분포 (Violin)")
        plt.grid(alpha=0.3)
        plt.tight_layout()
        plt.show()

        plt.figure(figsize=(6, 4))
        sns.boxplot(data=raw_df, x="target_label", y=col)
        plt.title(f"타겟별 {col} 분포 (Boxplot)")
        plt.grid(alpha=0.3)
        plt.tight_layout()
        plt.show()

# 페어플롯 (주요 변수만)
if numeric_cols:
    pair_cols = numeric_cols[:4] + ["target_label"]
    sns.pairplot(raw_df[pair_cols], hue="target_label", corner=True)
    plt.suptitle("주요 변수 페어플롯", y=1.02)
    plt.show()


## 인사이트 정리 (초안)

### 신용등급에 영향을 미칠 가능성이 높은 변수
- **연체 횟수 계열 변수**: 과거 연체 이력은 신용 위험의 직접 지표
- **부채비율/소득**: 상환 여력의 핵심 지표
- **나이/연체 패턴**: 위험군 분리에서 유의미할 수 있음

### 이상치 처리가 필요한 가능성이 높은 변수
- **월 소득, 부채비율**: 극단값이 많을 경우 모델 왜곡 가능
- **연체 횟수**: 비정상적으로 큰 값은 입력 오류 또는 특이 케이스 가능

### 파생 변수 필요성
- **소득 대비 부채 규모(비율)**, **연체 가중합** 등 파생 변수가 리스크 설명력을 높일 수 있음
- 규제 보고를 고려하면 **해석 가능한 파생 변수**가 유리

### 금융권 실무 비교
- 실제 신용평가에서는 **연체 이력 + 소득 안정성 + 부채비율**을 가장 중요하게 본다.
- 모델 성능뿐 아니라 **규제/감사 대응 가능성**(설명력)이 필수 요건이다.


## 전처리 파이프라인 구축 (sklearn Pipeline)

### 학습 포인트
- **Pipeline 장점**: 전처리+모델을 하나로 묶어 **재사용성**과 **데이터 유출 방지**를 확보
- **fit_transform vs transform**
  - `fit_transform`: 학습 데이터로 통계(평균/중앙값 등) 계산 + 변환
  - `transform`: 이미 계산된 통계를 적용해 변환만 수행
- **Java Builder 패턴 비교**: 단계별 설정을 체이닝해 하나의 객체(파이프라인)를 구성하는 방식과 유사

### 스케일링 선택 (금융 데이터 관점)
- **MinMaxScaler**: 이상치(outlier)에 민감 → 극단값이 있으면 스케일이 왜곡될 수 있음
- **StandardScaler**: 평균/표준편차 기반 → 금융 데이터의 분포 안정화에 더 적합한 경우가 많음
- 기본 선택은 **StandardScaler**, 필요 시 옵션으로 MinMax를 사용


In [ ]:
# =============================================================================
# 8. 전처리 파이프라인 함수 (결측치/스케일링/인코딩)
# =============================================================================
# 학습 목표: ColumnTransformer + Pipeline으로 전처리 흐름을 캡슐화한다
# Java 비유: Builder 패턴으로 전처리 모듈을 조립하는 방식과 유사

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler


def build_preprocess_pipeline(
    df: pd.DataFrame,
    target_col: str = "target",
    scaler_type: str = "standard",
) -> tuple[Pipeline, pd.DataFrame, pd.Series, list[str], list[str]]:
    """
    전처리 파이프라인을 구성하고 학습 데이터를 분리한다.

    Args:
        df (pd.DataFrame): 원본 데이터프레임
        target_col (str): 타겟 컬럼명
        scaler_type (str): 'standard' 또는 'minmax'

    Returns:
        tuple: (preprocess_pipeline, X_train, X_test, y_train, y_test) 반환
    """
    # 수치형/범주형 컬럼 분리
    numeric_features = df.select_dtypes(include=["int64", "float64"]).columns.tolist()
    categorical_features = df.select_dtypes(include=["object", "category"]).columns.tolist()

    if target_col in numeric_features:
        numeric_features.remove(target_col)

    if "target_label" in categorical_features:
        categorical_features.remove("target_label")

    # 스케일러 선택 (금융 데이터는 StandardScaler가 일반적으로 안정적)
    if scaler_type == "minmax":
        scaler = MinMaxScaler()
    else:
        scaler = StandardScaler()

    # 수치형 파이프라인: 결측치 -> 스케일링
    numeric_pipeline = Pipeline(
        steps=[
            ("imputer", SimpleImputer(strategy="median")),
            ("scaler", scaler),
        ]
    )

    # 범주형 파이프라인: 결측치 -> 원핫 인코딩
    categorical_pipeline = Pipeline(
        steps=[
            ("imputer", SimpleImputer(strategy="most_frequent")),
            ("encoder", OneHotEncoder(handle_unknown="ignore")),
        ]
    )

    # ColumnTransformer로 통합
    preprocess = ColumnTransformer(
        transformers=[
            ("num", numeric_pipeline, numeric_features),
            ("cat", categorical_pipeline, categorical_features),
        ],
        remainder="drop",
        sparse_threshold=0.3,  # 희소 행렬 비율 기준
    )

    # 입력/타겟 분리
    X = df.drop(columns=[target_col])
    y = df[target_col]

    # 데이터 분할 (80:20, 클래스 비율 유지)
    X_train, X_test, y_train, y_test = train_test_split(
        X,
        y,
        test_size=0.2,
        stratify=y,
        random_state=42,
    )

    # 최종 파이프라인 (전처리만 포함)
    preprocess_pipeline = Pipeline(
        steps=[
            ("preprocess", preprocess),
        ]
    )

    print("✅ 전처리 파이프라인 구성 완료")
    print("- 수치형 컬럼:", numeric_features)
    print("- 범주형 컬럼:", categorical_features)
    print("- 스케일러:", scaler.__class__.__name__)

    return preprocess_pipeline, X_train, X_test, y_train, y_test


# 파이프라인 생성 예시
preprocess_pipeline, X_train, X_test, y_train, y_test = build_preprocess_pipeline(
    raw_df,
    target_col="target",
    scaler_type="standard",
)

# fit_transform vs transform 차이 데모
X_train_prepared = preprocess_pipeline.fit_transform(X_train)
X_test_prepared = preprocess_pipeline.transform(X_test)

print("✅ 전처리 완료 (훈련/테스트)")
print("- X_train_prepared 타입:", type(X_train_prepared))
print("- X_test_prepared 타입:", type(X_test_prepared))


## 모델 학습 및 비교 (Baseline vs Ensemble)

### 실무 고려사항
- **신용평가에서는 Recall(미탐 방지)이 더 중요한 경우가 많다.**
  - Bad를 Good으로 잘못 분류하면 **부실 대출**로 직결
- **Precision도 중요**
  - Good 고객을 Bad로 오판하면 **우량 고객 이탈**과 기회 손실
- **False Negative(미탐)**: 부실 발생 → 손실/대손충당금 증가
- **False Positive(오탐)**: 고객 이탈 → 수익 기회 손실
- **규제 관점**: 해석 가능성(Logistic) + 성능(RandomForest/XGBoost) 균형 필요

### 모델 특징 요약
- **Logistic Regression**: 해석 용이, 규제 대응에 유리, 성능은 제한적
- **Random Forest**: 비선형 패턴 학습, 변수 중요도 해석 가능, 학습 시간 증가
- **XGBoost**: 성능 강점, 하이퍼파라미터 민감, 규제/설명력 보완 필요


In [ ]:
# =============================================================================
# 9. 모델 학습 및 평가 유틸리티
# =============================================================================
# 학습 목표: 여러 모델을 동일 기준으로 학습/평가한다
# Java 비유: 공통 인터페이스로 구현체를 교체하며 성능 비교하는 방식과 유사

import time
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    roc_curve,
)
from sklearn.model_selection import StratifiedKFold


def evaluate_model(name: str, model_pipeline: Pipeline, X_train, X_test, y_train, y_test) -> dict:
    """
    모델 학습 후 주요 지표를 계산하고 ROC/Confusion Matrix를 출력한다.
    """
    start_time = time.time()
    model_pipeline.fit(X_train, y_train)
    train_time = time.time() - start_time

    y_pred = model_pipeline.predict(X_test)

    # ROC-AUC 계산을 위해 확률 예측 필요
    if hasattr(model_pipeline, "predict_proba"):
        y_proba = model_pipeline.predict_proba(X_test)[:, 1]
    else:
        y_proba = None

    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc_score = roc_auc_score(y_test, y_proba) if y_proba is not None else np.nan

    print(f"\n✅ {name} 평가 결과")
    print(f"- Accuracy: {accuracy:.4f}")
    print(f"- Precision: {precision:.4f}")
    print(f"- Recall: {recall:.4f}")
    print(f"- F1-Score: {f1:.4f}")
    print(f"- ROC-AUC: {auc_score:.4f}")
    print(f"- 학습 시간: {train_time:.2f}초")

    # Confusion Matrix 시각화
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(4, 3))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
    plt.title(f"{name} Confusion Matrix")
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.tight_layout()
    plt.show()

    # ROC Curve
    if y_proba is not None:
        fpr, tpr, _ = roc_curve(y_test, y_proba)
        plt.figure(figsize=(5, 4))
        plt.plot(fpr, tpr, label=f"AUC={auc_score:.3f}")
        plt.plot([0, 1], [0, 1], linestyle="--", color="gray")
        plt.title(f"{name} ROC Curve")
        plt.xlabel("False Positive Rate")
        plt.ylabel("True Positive Rate")
        plt.legend(loc="lower right")
        plt.grid(alpha=0.3)
        plt.tight_layout()
        plt.show()

    # Classification Report
    print("\n📌 Classification Report")
    print(classification_report(y_test, y_pred))

    return {
        "model": name,
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "auc": auc_score,
        "train_time_sec": train_time,
    }


def cross_validate_model(name: str, model_pipeline: Pipeline, X, y) -> dict:
    """
    StratifiedKFold 5-fold 교차 검증 수행
    """
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    scores = {"accuracy": [], "precision": [], "recall": [], "f1": [], "auc": []}

    for fold, (train_idx, test_idx) in enumerate(skf.split(X, y), start=1):
        X_train_cv, X_test_cv = X.iloc[train_idx], X.iloc[test_idx]
        y_train_cv, y_test_cv = y.iloc[train_idx], y.iloc[test_idx]

        model_pipeline.fit(X_train_cv, y_train_cv)
        y_pred_cv = model_pipeline.predict(X_test_cv)

        if hasattr(model_pipeline, "predict_proba"):
            y_proba_cv = model_pipeline.predict_proba(X_test_cv)[:, 1]
        else:
            y_proba_cv = None

        scores["accuracy"].append(accuracy_score(y_test_cv, y_pred_cv))
        scores["precision"].append(precision_score(y_test_cv, y_pred_cv))
        scores["recall"].append(recall_score(y_test_cv, y_pred_cv))
        scores["f1"].append(f1_score(y_test_cv, y_pred_cv))
        scores["auc"].append(roc_auc_score(y_test_cv, y_proba_cv) if y_proba_cv is not None else np.nan)

        print(f"✅ {name} Fold {fold} 완료")

    # 평균/표준편차 요약
    summary = {metric: (np.mean(values), np.std(values)) for metric, values in scores.items()}
    print(f"\n📌 {name} 교차 검증 평균/표준편차")
    for metric, (mean_val, std_val) in summary.items():
        print(f"- {metric}: {mean_val:.4f} ± {std_val:.4f}")

    return summary


In [ ]:
# =============================================================================
# 10. 모델 학습 및 비교 실행
# =============================================================================
# 학습 목표: baseline과 앙상블 모델의 성능을 비교한다
# Java 비유: 여러 구현체를 같은 인터페이스로 평가하는 방식과 유사

# 모델 정의
logistic_model = LogisticRegression(max_iter=1000)
rf_model = RandomForestClassifier(n_estimators=200, random_state=42)

models = {
    "Logistic Regression": logistic_model,
    "Random Forest": rf_model,
}

# XGBoost는 설치된 경우만 추가
if xgb is not None:
    xgb_model = xgb.XGBClassifier(
        n_estimators=200,
        max_depth=4,
        learning_rate=0.1,
        subsample=0.8,
        colsample_bytree=0.8,
        eval_metric="logloss",
        random_state=42,
    )
    models["XGBoost"] = xgb_model
else:
    print("⚠️ XGBoost 모델은 설치되지 않아 스킵합니다.")

# 공통 전처리 파이프라인 재사용
results = []
cv_results = {}

for name, model in models.items():
    full_pipeline = Pipeline(
        steps=[
            ("preprocess", preprocess_pipeline.named_steps["preprocess"]),
            ("model", model),
        ]
    )

    # 모델 평가
    result = evaluate_model(name, full_pipeline, X_train, X_test, y_train, y_test)
    results.append(result)

    # 교차 검증
    cv_summary = cross_validate_model(name, full_pipeline, X_train, y_train)
    cv_results[name] = cv_summary

# 모델 비교 테이블
results_df = pd.DataFrame(results)
print("\n✅ 모델 비교 테이블")
print(results_df)

# 성능 기준: AUC 우선, 그다음 F1
results_df_sorted = results_df.sort_values(by=["auc", "f1"], ascending=False)
print("\n✅ 성능 우수 모델 (AUC, F1 기준)")
print(results_df_sorted[["model", "auc", "f1", "precision", "recall"]])


## 평가 지표 해석 (금융 실무 관점)

- **Accuracy**: 전체 맞춘 비율, 불균형 데이터에서는 과대평가 위험
- **Precision**: Bad로 예측한 것 중 실제 Bad 비율 → 불필요한 거절 최소화에 중요
- **Recall**: 실제 Bad 중 Bad로 잡아낸 비율 → 부실 대출 방지에 핵심
- **F1-Score**: Precision/Recall 균형 지표 → 리스크/수익 균형 판단
- **ROC-AUC**: 임계값에 독립적인 분류 성능 → 모델 비교에 유용

### 어떤 지표가 더 중요한가?
- **리스크 최소화 관점**: Recall 우선 (부실 대출 방지)
- **우량 고객 확보 관점**: Precision도 중요 (과도한 거절 방지)

### 규제/감사 관점 모델 선택 기준
- **설명 가능성**: Logistic Regression 우위
- **성능 중심**: XGBoost/RandomForest 우위
- **실무 균형**: 성능 + 설명력 + 안정성의 균형이 핵심


## 모델 최적화 및 해석 (배포 관점 포함)

### 실무 활용
- **Feature Importance**는 신용정책의 우선순위(예: 연체 이력, 부채비율 기준)를 설계할 때 참고 지표로 활용
- 중요 변수가 정책과 충돌할 경우, **규제/감사 대응 가능성**을 위해 해석 가능한 기준으로 재정의 필요

### 금융 규제 관점
- 모델 성능만큼 **설명 가능성(해석력)**이 중요
- 규제기관은 “왜 이 고객이 거절되었는가?”를 설명할 수 있어야 함

### 배포 고려사항
- **데이터 드리프트** 모니터링(분포 변화 감지)
- **재학습 주기** 관리(월/분기)
- **모델 버전 관리** 및 승인 프로세스
- **리스크 한도**와의 연계(모델 결과를 정책에 매핑)


In [ ]:
# =============================================================================
# 11. 하이퍼파라미터 튜닝 (GridSearchCV)
# =============================================================================
# 학습 목표: 모델 성능을 개선하기 위해 최적 파라미터를 탐색한다
# Java 비유: 설정값을 조합해 최적 성능을 찾는 튜닝 실험과 유사

from sklearn.model_selection import GridSearchCV
import joblib

# 공통 전처리 + 모델 파이프라인 생성 함수

def build_full_pipeline(model) -> Pipeline:
    return Pipeline(
        steps=[
            ("preprocess", preprocess_pipeline.named_steps["preprocess"]),
            ("model", model),
        ]
    )

# Random Forest GridSearch
rf_param_grid = {
    "model__n_estimators": [100, 200, 300],
    "model__max_depth": [10, 20, 30, None],
    "model__min_samples_split": [2, 5, 10],
}

rf_grid = GridSearchCV(
    estimator=build_full_pipeline(RandomForestClassifier(random_state=42)),
    param_grid=rf_param_grid,
    scoring="roc_auc",
    cv=5,
    n_jobs=-1,
    verbose=1,
)

rf_grid.fit(X_train, y_train)
print("✅ Random Forest 최적 파라미터:", rf_grid.best_params_)
print("✅ Random Forest 최고 AUC:", rf_grid.best_score_)

# XGBoost GridSearch (설치된 경우만)
if xgb is not None:
    xgb_param_grid = {
        "model__learning_rate": [0.01, 0.1, 0.3],
        "model__max_depth": [3, 5, 7],
        "model__n_estimators": [100, 200, 300],
    }

    xgb_grid = GridSearchCV(
        estimator=build_full_pipeline(
            xgb.XGBClassifier(
                eval_metric="logloss",
                random_state=42,
                subsample=0.8,
                colsample_bytree=0.8,
            )
        ),
        param_grid=xgb_param_grid,
        scoring="roc_auc",
        cv=5,
        n_jobs=-1,
        verbose=1,
    )

    xgb_grid.fit(X_train, y_train)
    print("✅ XGBoost 최적 파라미터:", xgb_grid.best_params_)
    print("✅ XGBoost 최고 AUC:", xgb_grid.best_score_)
else:
    xgb_grid = None
    print("⚠️ XGBoost가 설치되지 않아 튜닝을 스킵합니다.")


In [ ]:
# =============================================================================
# 12. Feature Importance (Random Forest)
# =============================================================================
# 학습 목표: 모델이 중요하게 보는 변수를 파악한다
# Java 비유: 중요도가 높은 필드를 우선적으로 검토하는 코드 리뷰와 유사

# GridSearch 결과가 있으면 최적 모델 사용, 없으면 기본 RF 사용
best_rf_model = rf_grid.best_estimator_ if "rf_grid" in globals() else None
if best_rf_model is None:
    best_rf_model = build_full_pipeline(RandomForestClassifier(random_state=42))
    best_rf_model.fit(X_train, y_train)

# 전처리된 feature 이름 추출
preprocess = best_rf_model.named_steps["preprocess"]
feature_names = preprocess.get_feature_names_out()

# 모델의 feature_importances_ 추출
rf_model = best_rf_model.named_steps["model"]
importances = rf_model.feature_importances_

feature_importance_df = pd.DataFrame(
    {"feature": feature_names, "importance": importances}
).sort_values(by="importance", ascending=False)

# 상위 10개 중요 변수
top_features = feature_importance_df.head(10)
print("✅ 상위 10개 중요 변수")
print(top_features)

plt.figure(figsize=(8, 5))
sns.barplot(data=top_features, x="importance", y="feature")
plt.title("Random Forest 중요 변수 Top 10")
plt.tight_layout()
plt.show()


In [ ]:
# =============================================================================
# 13. SHAP Values (선택)
# =============================================================================
# 학습 목표: 개별 예측의 원인을 설명할 수 있다
# Java 비유: 로깅/트레이싱으로 의사결정 근거를 추적하는 방식과 유사

try:
    import shap

    # SHAP 계산용 데이터 샘플링 (속도 개선)
    sample_size = min(500, X_train.shape[0])
    X_sample = X_train.sample(n=sample_size, random_state=42)

    # 전처리 변환 적용
    X_sample_prepared = preprocess.transform(X_sample)

    # Tree 기반 모델에 적합한 Explainer 사용
    explainer = shap.TreeExplainer(rf_model)
    shap_values = explainer.shap_values(X_sample_prepared)

    # 요약 플롯
    shap.summary_plot(shap_values[1], X_sample_prepared, feature_names=feature_names)

    # 개별 예측 설명 (첫 번째 샘플)
    shap.force_plot(
        explainer.expected_value[1],
        shap_values[1][0],
        X_sample_prepared[0],
        feature_names=feature_names,
        matplotlib=True,
    )

except ModuleNotFoundError:
    print("⚠️ shap 라이브러리가 설치되어 있지 않습니다. (pip install shap)")


In [ ]:
# =============================================================================
# 14. 최종 모델 저장
# =============================================================================
# 학습 목표: 최적 모델을 파일로 저장해 배포/재사용한다
# Java 비유: 학습된 객체를 직렬화하여 재사용하는 방식과 유사

# 최적 모델 선택: XGBoost > RandomForest 기준 (가능한 경우)
if xgb_grid is not None:
    best_model_pipeline = xgb_grid.best_estimator_
    best_model_name = "XGBoost"
else:
    best_model_pipeline = rf_grid.best_estimator_
    best_model_name = "RandomForest"

model_path = Path.cwd() / "best_credit_model.pkl"
joblib.dump(best_model_pipeline, model_path)
print(f"✅ 최종 모델 저장 완료: {model_path} ({best_model_name})")
